In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import seaborn as sns
sns.set_theme(style="darkgrid")


In [2]:
df = pd.read_csv(r"C:\Users\brvn\Documents\github\hc-demand-forecasting\data\interim\oftalmo_ts.csv")
df.head(), df.shape

(                          unique_id          ds  y
 0  0201010097 BIOPSIA DE CONJUNTIVA  2017-10-01  0
 1  0201010097 BIOPSIA DE CONJUNTIVA  2017-11-01  0
 2  0201010097 BIOPSIA DE CONJUNTIVA  2017-12-01  2
 3  0201010097 BIOPSIA DE CONJUNTIVA  2018-01-01  0
 4  0201010097 BIOPSIA DE CONJUNTIVA  2018-02-01  0,
 (5475, 3))

Filtrar no dataframe os procedimentos sub utilizados.

In [ ]:
def filter_dataframe(df)-> pd.DataFrame:
    ref_date = pd.to_datetime('2022-09-01')
    
    filtered_df = df[df['ds'] >= ref_date]
    
    inactives = filtered_df.groupby('unique_id', observed=False)['y'].all()[lambda x: x == 0].index.tolist()
    print(inactives)
    actives_df = df[~df['unique_id'].isin(inactives)]
    return actives_df


In [ ]:
df = filter_dataframe(df)


In [ ]:
# Identify and remove the COVID-19 period
covid_start_date = pd.to_datetime('2020-03-01')
covid_end_date = pd.to_datetime('2021-06-01')
df = df.loc[~((df['ds'] >= covid_start_date) & (df['ds'] <= covid_end_date))]


In [ ]:
df = df.set_index('unique_id')

In [ ]:
from statsforecast.models import (
    AutoARIMA,
    AutoETS
)
from statsforecast import StatsForecast

models= [
    AutoARIMA(season_length=12),
    AutoETS(season_length=12)
]

# Instantiate StatsForecast class as sf
sf = StatsForecast(
    df=df, 
    models=models,
    freq='MS',
    verbose=True,   
)

In [ ]:
forecast_df = sf.forecast(h=12, level=[90])


In [ ]:
sf.plot(df, forecast_df, engine='matplotlib')

In [ ]:
crossvaldation_df = sf.cross_validation(
    df=df,
    h=12,
    step_size=12,
    n_windows=2,
    sort_df=True,
  )

In [ ]:
crossvaldation_df.head()

In [ ]:
from utilsforecast.losses import mse
from utilsforecast.evaluation import evaluate

def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['unique_id', 'ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    # Calculate loss for every unique_id and cutoff.    
    for cutoff in df['cutoff'].unique():
        eval_ = evaluate(df[df['cutoff'] == cutoff], metrics=[metric], models=models)
        evals.append(eval_)
    evals = pd.concat(evals)
    evals = evals.groupby('unique_id', observed=False).mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

In [ ]:
evaluation_df = evaluate_cross_validation(crossvaldation_df.reset_index(), mse)

evaluation_df.head()

In [ ]:
#Criando um sumário dos melhores modelos
summary_df = evaluation_df.groupby('best_model').size().sort_values().to_frame()

summary_df.reset_index().columns = ["Model", "Nr. of unique_ids"]

# autoets_ids = evaluation_df.query('best_model == "AutoETS"').index
# autoarima_ids = evaluation_df.query('best_model == "AutoARIMA"').index

In [ ]:
autoarima_ids = evaluation_df.query('best_model == "AutoARIMA"').index
sf.plot(df=df,forecasts_df=forecast_df, unique_ids=autoarima_ids, engine='matplotlib')

In [ ]:
autoets_ids = evaluation_df.query('best_model == "AutoETS"').index
sf.plot(df=df, forecasts_df=forecast_df, unique_ids=autoets_ids, engine='matplotlib')

In [ ]:
def get_best_model_forecast(forecasts_df, evaluation_df):
    df = forecasts_df.set_index('ds', append=True).stack().to_frame().reset_index(level=2) # Wide to long 
    df.columns = ['model', 'best_model_forecast'] 
    df = df.join(evaluation_df[['best_model']])
    df = df.query('model.str.replace("-lo-90|-hi-90", "", regex=True) == best_model').copy()
    df.loc[:, 'model'] = [model.replace(bm, 'best_model') for model, bm in zip(df['model'], df['best_model'])]
    df = df.drop(columns='best_model').set_index('model', append=True).unstack()
    df.columns = df.columns.droplevel()
    df = df.reset_index(level=1)
    return df

In [ ]:
prod_forecasts_df = get_best_model_forecast(forecast_df, evaluation_df)
sf.plot(df, prod_forecasts_df, level=[90], engine='matplotlib')

In [ ]:
prod_forecasts_df.to_csv("covid-forecasted-df.csv", float_format="%d")

In [ ]:
# Visualize the final time series
plt.figure(figsize=(10, 6))
sns.lineplot(
    x="ds", y="y",
    hue="unique_id", 
    # style="unique_id",
    data=df, 
    legend=False
)